In [61]:
import scipy.io as sio
from scipy.misc import imread
from preprocess.normalize import preprocess_signature
import tf_signet
from tf_cnn_model import TF_CNNModel
import tensorflow as tf
import numpy as np
import pandas as pd
import sys
import os
import scipy.io
from find_largest_image import find_largest
import tqdm 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from xgboost import XGBClassifier
import random
from numpy.random import choice
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [2]:
data_folder = 'C:\\Users\\Mert\\Documents\\GitHub\\sigver_bmg\\data\\downloaded_pp_features\\gpds_signet\\validation'

In [3]:
data_f=pd.read_csv(os.path.join(data_folder,'data_features.csv'))
visual_f=pd.read_csv(os.path.join(data_folder,'visual_features.csv'))

In [4]:
data_f.head()

,user_id,sig_id,fakeness
0,352,1,True
1,352,2,True
2,352,3,True
3,352,4,True
4,352,5,True


In [5]:
visual_f.head()

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0.000000,0.000000,0.0,1.370572,0.000000,0.000000,0.0,0.149738,0.323547,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,1.186365,0.331796,0.356497,0.0
1,0.160537,0.000000,0.0,1.169396,0.239608,0.000000,0.0,0.000000,1.208937,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,1.141376,0.893188,0.000000,0.0
2,0.393000,0.223869,0.0,0.195907,0.000000,0.000000,0.0,0.060111,0.088993,0.194043,...,0.0,0.0,0.0,0.0,0.0,0.000000,1.038160,0.215892,0.050661,0.0
3,0.280439,0.609959,0.0,0.392170,0.000000,0.568858,0.0,0.000000,0.246439,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.138333,1.180603,0.000000,0.000000,0.0
4,0.351282,0.000000,0.0,0.000000,0.000000,0.940088,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.398407,1.215286,0.000000,0.000000,0.0


In [6]:
data_f_gen = data_f.loc[data_f['fakeness']==False]
data_f_fake = data_f.loc[data_f['fakeness']==True]
data_f_gen_12 = data_f_gen.loc[data_f_gen['sig_id'].isin(np.arange(1,13))]
data_f_gen_valid_12 = data_f_gen.loc[data_f_gen['sig_id'].isin(np.arange(13,25))]

In [60]:
scores_sum = 0
run_count = 0
gens_sum = 0
gens_r_count = 0
fakes_sum = 0
fakes_r_count = 0
randoms_sum = 0
randoms_r_count = 0
for user_id in tqdm.tqdm(data_f['user_id'].unique(), ascii=True):
    y_train = data_f_gen_12['user_id']==user_id
    X_train = visual_f.loc[y_train.index]
    class_weight = (data_f_gen_12.shape[0]-data_f_gen_12[y_train].shape[0])/data_f_gen_12[y_train].shape[0]
    clf = XGBClassifier(tree_method='gpu_hist',verbosity=2, scale_pos_weight=50*float(class_weight), objective='reg:logistic')
    # clf = SVC(class_weight='balanced',gamma='scale') # gens avg: 0.8593333333333333 fakes_avg: 0.9433333333333331 randoms_avg: 1.0 cumulative_avg: 0.9342222222222221
    clf.fit(X_train, y_train)
    y_valid_fakes = data_f_fake.loc[data_f_fake['user_id']==user_id]
    y_valid_f = len(y_valid_fakes.index)*[False]
    X_valid_f = visual_f.loc[y_valid_fakes.index]
    score = clf.score(X_valid_f,y_valid_f)
    fakes_sum += score
    fakes_r_count += 1 
    # print(score)
    scores_sum += score
    run_count += 1
    y_valid_gens = data_f_gen_valid_12.loc[data_f_gen_valid_12['user_id']==user_id]
    y_valid_g = len(y_valid_gens.index)*[True]
    X_valid_g = visual_f.loc[y_valid_gens.index]
    score = clf.score(X_valid_g,y_valid_g)
    gens_sum += score
    gens_r_count += 1 
    # print(score)
    scores_sum += score
    run_count += 1
    '''
    y_valid_rand_fakes = data_f_gen_valid_12.loc[choice(data_f_gen_valid_12.loc[data_f_gen_valid_12['user_id']!=user_id].index,10)]
    y_valid_rf = len(y_valid_rand_fakes.index)*[False]
    X_valid_rf = visual_f.loc[y_valid_rand_fakes.index]
    score = clf.score(X_valid_rf,y_valid_rf)
    randoms_sum += score
    randoms_r_count += 1 
    # print(score)
    scores_sum += score
    run_count += 1
    # print(scores_sum/run_count)
    '''
    print ("gens avg:", float(gens_sum)/gens_r_count, "fakes_avg:", float(fakes_sum)/fakes_r_count, 
        "cumulative_avg:", (float(gens_sum)/gens_r_count)/2+ 
        (float(fakes_sum)/fakes_r_count)/2)
print(scores_sum/run_count)
print("gens avg:", float(gens_sum)/gens_r_count, "fakes_avg:", float(fakes_sum)/fakes_r_count, 
    "cumulative_avg:", (float(gens_sum)/gens_r_count)/2+ 
    (float(fakes_sum)/fakes_r_count)/2)





  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

gens avg: 1.0 fakes_avg: 0.13333333333333333 cumulative_avg: 0.5666666666666667






  2%|#6                                                                                 | 1/50 [00:01<01:18,  1.61s/it]

gens avg: 1.0 fakes_avg: 0.31666666666666665 cumulative_avg: 0.6583333333333333






  4%|###3                                                                               | 2/50 [00:03<01:17,  1.61s/it]

...

gens avg: 0.8231292517006802 fakes_avg: 0.7544217687074828 cumulative_avg: 0.7887755102040814






 98%|################################################################################3 | 49/50 [01:21<00:01,  1.58s/it]

gens avg: 0.8216666666666665 fakes_avg: 0.7586666666666665 cumulative_avg: 0.7901666666666665






100%|##################################################################################| 50/50 [01:23<00:00,  1.55s/it]

0.7901666666666667
gens avg: 0.8216666666666665 fakes_avg: 0.7586666666666665 cumulative_avg: 0.7901666666666665


In [50]:
clf.predict(X_valid_f)

array([False,  True,  True,  True, False, False, False,  True, False,
        True,  True,  True,  True,  True,  True, False, False, False,
        True, False,  True, False, False, False,  True, False, False,
       False,  True,  True])

In [58]:
list(map(bool,np.argmax(clf.predict_proba(X_valid_f), axis=1)))

[False,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 True]